In [1]:
from pprint import pprint
from bigbio.dataloader import BigBioConfigHelpers
from bigbio.utils.constants import Tasks

# BigBioConfigHelpers

Start by creating an instance of BigBioConfigHelpers. This will help locate and filter datasets available in the BigBIO package. 

In [2]:
conhelps = BigBioConfigHelpers()
print("found {} dataset configs from {} datasets".format(
    len(conhelps),
    len(conhelps.available_dataset_names)
))

found 453 dataset configs from 127 datasets


Each dataset has at least one source config and at least one bigbio config. Source configs attempt to preserve the original structure of the dataset while bigbio configs are normalized into one of several bigbio [task schemas](https://github.com/bigscience-workshop/biomedical/blob/master/task_schemas.md). Some datasets have several source configs and/or bigbio configs (e.g. multi-lingual datasets or datasets supporting multiple cross-validation folds). This is why the number of configs is greater than twice the number of datasets.

### Examine One Helper

conhelps is list-like and elements can be accesed with integer indices. Lets examine one.

In [3]:
pprint(conhelps[0])

BigBioConfigHelper(script='/home/galtay/repos/biomedical/bigbio/biodatasets/an_em/an_em.py',
                   dataset_name='an_em',
                   tasks=[<Tasks.NAMED_ENTITY_RECOGNITION: 'NER'>,
                          <Tasks.COREFERENCE_RESOLUTION: 'COREF'>,
                          <Tasks.RELATION_EXTRACTION: 'RE'>],
                   languages=[<Lang.EN: 'English'>],
                   config=BigBioConfig(name='an_em_source',
                                       version=1.0.4,
                                       data_dir=None,
                                       data_files=None,
                                       description='AnEM source schema',
                                       schema='source',
                                       subset_id='an_em'),
                   is_local=False,
                   is_pubmed=True,
                   is_bigbio_schema=False,
                   bigbio_schema_caps=None,
                   is_large=False,
             

### Show All Available Datasets

In [4]:
print(conhelps.available_dataset_names)

['an_em', 'anat_em', 'ask_a_patient', 'bc5cdr', 'bc7_litcovid', 'bio_sim_verb', 'bio_simlex', 'bioasq_2021_mesinesp', 'bioasq_task_b', 'bioasq_task_c_2017', 'bioinfer', 'biology_how_why_corpus', 'biomrc', 'bionlp_shared_task_2009', 'bionlp_st_2011_epi', 'bionlp_st_2011_ge', 'bionlp_st_2011_id', 'bionlp_st_2011_rel', 'bionlp_st_2013_cg', 'bionlp_st_2013_ge', 'bionlp_st_2013_gro', 'bionlp_st_2013_pc', 'bionlp_st_2019_bb', 'biored', 'biorelex', 'bioscope', 'biosses', 'cadec', 'cantemist', 'cas', 'cellfinder', 'chebi_nactem', 'chemdner', 'chemprot', 'chia', 'citation_gia_test_collection', 'codiesp', 'cord_ner', 'ctebmsp', 'ddi_corpus', 'diann_iber_eval', 'distemist', 'ebm_pico', 'ehr_rel', 'essai', 'euadr', 'evidence_inference', 'gad', 'genetag', 'genia_ptm_event_corpus', 'genia_relation_corpus', 'genia_term_corpus', 'geokhoj_v1', 'gnormplus', 'hallmarks_of_cancer', 'hprd50', 'iepa', 'jnlpba', 'linnaeus', 'lll', 'mantra_gsc', 'mayosrs', 'med_qa', 'medal', 'meddialog', 'meddocan', 'medhop',

### Show Helpers for Specific Dataset

We can also get the helpers for a specific dataset using the dataset name. 

In [5]:
bc5cdr_helpers = conhelps.for_dataset("bc5cdr")
print(len(bc5cdr_helpers))
pprint(bc5cdr_helpers[0].config)
pprint(bc5cdr_helpers[1].config)

2
BigBioConfig(name='bc5cdr_source',
             version=1.5.16,
             data_dir=None,
             data_files=None,
             description='BC5CDR source schema',
             schema='source',
             subset_id='bc5cdr')
BigBioConfig(name='bc5cdr_bigbio_kb',
             version=1.0.0,
             data_dir=None,
             data_files=None,
             description='BC5CDR simplified BigBio schema',
             schema='bigbio_kb',
             subset_id='bc5cdr')


# Loading Datasets

Each config helper provides a wrapper to the [load_dataset](https://huggingface.co/docs/datasets/v2.2.1/en/package_reference/loading_methods#datasets.load_dataset) function from Huggingface's [datasets](https://huggingface.co/docs/datasets/) package. This wrapper will automatically populate the first two arguments of load_dataset,

* path: path to the dataloader script
* name: name of the dataset configuration

If you have a specific dataset and config in mind, you can,

* fetch the helper from conhelps with the for_config_name method
* load the dataset using the load_dataset wrapper

In [6]:
bc5cdr_source = conhelps.for_config_name("bc5cdr_source").load_dataset()
bc5cdr_bigbio = conhelps.for_config_name("bc5cdr_bigbio_kb").load_dataset()

Reusing dataset bc5cdr_dataset (/home/galtay/.cache/huggingface/datasets/bc5cdr_dataset/bc5cdr_source/1.5.16/f01f16ea9b65ead985bedadf7335195c32297c8f1b09417fc607b102a6757d6f)


  0%|          | 0/3 [00:00<?, ?it/s]

Reusing dataset bc5cdr_dataset (/home/galtay/.cache/huggingface/datasets/bc5cdr_dataset/bc5cdr_bigbio_kb/1.0.0/f01f16ea9b65ead985bedadf7335195c32297c8f1b09417fc607b102a6757d6f)


  0%|          | 0/3 [00:00<?, ?it/s]

This wrapper function will pass through any other kwargs you may need to use. For example data_dir for datasets that are not public,

In [7]:
# note this will not work unless you have the n2c2 dataset locally

#n2c2_2011_source = (
#    conhelps.
#    for_config_name("n2c2_2011_source").
#    load_dataset(data_dir="/path/to/n2c2_2011/data")
#)

# Filter and Load Multiple Datasets

You can use any attribute of a BigBioConfigHelper to filter the collection. Here are some examples,

### BigBIO schema datasets that are public and not "large"

In [8]:
bb_public_helpers = conhelps.filtered(
    lambda x:
        x.is_bigbio_schema
        and not x.is_local
        and not x.is_large
)

### Source schema for n2c2 datasets

In [9]:
n2c2_source_helpers = conhelps.filtered(
    lambda x:
        x.dataset_name.startswith("n2c2")
        and not x.is_bigbio_schema
)

### BigBIO schema datasets that are public and support textual entailment

In [10]:
entailment_helpers = conhelps.filtered(
    lambda x:
        x.is_bigbio_schema
        and not x.is_local
        and Tasks.TEXTUAL_ENTAILMENT in x.tasks
)

### Loading filtered datasets

Note that the `filtered` method returns another instance of `BigBioConfigHelpers`. This means you can iterate over any of the helpers defined above and load all of the datasets. 

In [11]:
bb_public_datasets = {
    helper.config.name: helper.load_dataset()
    for helper in bb_public_helpers
    if helper.dataset_name != 'bioasq_2021_mesinesp' # hosting is spotty
}

Reusing dataset an_em_dataset (/home/galtay/.cache/huggingface/datasets/an_em_dataset/an_em_bigbio_kb/1.0.0/7a60ad16ca5e51e4ccc8d8de0169364df987f7c94f307050e0ddab5473d4f13b)


  0%|          | 0/3 [00:00<?, ?it/s]

Reusing dataset anat_em_dataset (/home/galtay/.cache/huggingface/datasets/anat_em_dataset/anat_em_bigbio_kb/1.0.0/5fdfe355922d9812744a70efd7fe46a322f9dba70f3ed29464e0c1009e4d04a1)


  0%|          | 0/3 [00:00<?, ?it/s]

Reusing dataset ask_a_patient (/home/galtay/.cache/huggingface/datasets/ask_a_patient/ask_a_patient_bigbio_kb/1.0.0/b3a42779cd6f10ed9a7d30be59cf32b0046c9d6e259e93c72fefbfd4f9716ef4)


...

  0%|          | 0/1 [00:00<?, ?it/s]

Reusing dataset umnsrs_dataset (/home/galtay/.cache/huggingface/datasets/umnsrs_dataset/umnsrs_relatedness_bigbio_pairs/1.0.0/8c7002a472ad988e442babd6c70bd30d98eede363282d56922ace6db655cbf3a)


  0%|          | 0/1 [00:00<?, ?it/s]

Reusing dataset verspoor2013_dataset (/home/galtay/.cache/huggingface/datasets/verspoor2013_dataset/verspoor_2013_bigbio_kb/1.0.0/7494b1d9332951566d7e01281b0e7ced1634721918510ce9fb40795490119ee0)


  0%|          | 0/1 [00:00<?, ?it/s]

# Dataset Metadata

Each BigBioConfigHelper provides a get_metadata method that will calculate schema specific metadata for configs implementing a BigBIO schema. For example,

In [12]:
conhelps.for_config_name('bc5cdr_bigbio_kb').get_metadata()

Reusing dataset bc5cdr_dataset (/home/galtay/.cache/huggingface/datasets/bc5cdr_dataset/bc5cdr_bigbio_kb/1.0.0/f01f16ea9b65ead985bedadf7335195c32297c8f1b09417fc607b102a6757d6f)


  0%|          | 0/3 [00:00<?, ?it/s]

{'train': BigBioKbMetadata(samples_count=500, passages_count=1000, passages_char_count=652177, passages_type_counter={'title': 500, 'abstract': 500}, entities_count=9570, entities_normalized_count=9599, entities_type_counter={'Chemical': 5207, 'Disease': 4363}, entities_db_name_counter={'MESH': 9599}, entities_unique_db_ids_count=1328, events_count=0, events_type_counter={}, events_arguments_count=0, events_arguments_role_counter={}, coreferences_count=0, relations_count=15072, relations_type_counter={'CID': 15072}, relations_db_name_counter={}, relations_unique_db_ids_count=0),
 'test': BigBioKbMetadata(samples_count=500, passages_count=1000, passages_char_count=676751, passages_type_counter={'title': 500, 'abstract': 500}, entities_count=9928, entities_normalized_count=9919, entities_type_counter={'Chemical': 5394, 'Disease': 4534}, entities_db_name_counter={'MESH': 9919}, entities_unique_db_ids_count=1315, events_count=0, events_type_counter={}, events_arguments_count=0, events_argu